# ONNX tutorial with TensorFlow

## Install necessary libraries

Clone the repo with notebooks and corresponding data. 

In [ ]:
!git clone https://github.com/TurboML-Inc/colab-notebooks.git

Set up the environment and install TurboML's SDK. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!bash colab-notebooks/install_turboml.sh

The kernel should now be restarted with TurboML's SDK installed.

In [ ]:
cd colab-notebooks

Login to your TurboML instance.

In [ ]:
import pandas as pd
import turboml as tb
tb.init(backend_url=BACKEND_URL, api_key=API_KEY)

In [ ]:
!pip install -U tensorflow tf2onnx onnx==1.14.1 scikit-learn

## PyTorch - Standard Model Training

The following blocks of code define a standard tensorflow training code. This is completely independent of TurboML. 

In [ ]:
import tf2onnx.convert
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
transactions = pd.read_csv("data/transactions.csv").reset_index()
labels = pd.read_csv("data/labels.csv").reset_index()

In [ ]:
joined_df = pd.merge(transactions, labels, on="index", how="right")
joined_df

In [ ]:
X = joined_df.drop("is_fraud", axis=1)
numerical_fields = [
    "transactionAmount",
    "localHour",
    "isProxyIP",
    "digitalItemCount",
    "physicalItemCount",
]

feats = X[numerical_fields]
targets = joined_df["is_fraud"].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    feats, targets, test_size=0.2, random_state=42
)

In [ ]:
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation="relu"))
model.add(Dense(2, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

num_epochs = 10
model.fit(X_train, y_train, epochs=num_epochs, batch_size=64, verbose=1)

In [ ]:
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

## Export model to ONNX format

Exporting a model to ONNX format depends on the framework. Tutorials for different frameworks can be found at https://github.com/onnx/tutorials#converting-to-onnx-format

In [ ]:
onnx_model_path = "tensorflow_model.onnx"
input_signature = [
    tf.TensorSpec([1, len(numerical_fields)], tf.float32, name="keras_tensor")
]
model.output_names = ["output"]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)

onnx_model_bytes = onnx_model.SerializeToString()

## Create an ONNX model with TurboML

Now that we've converted the model to ONNX format, we can deploy it with TurboML. 

In [ ]:

try:
    transactions = tb.PandasDataset(
        dataset_name="transactions_onxx_tf",
        key_field="index",
        dataframe=transactions,
        upload=True,
    )
except:
    transactions = tb.PandasDataset(dataset_name="transactions_onxx_tf")

try:
    labels = tb.PandasDataset(
        dataset_name="labels", key_field="index", dataframe=labels, upload=True
    )
except:
    labels = tb.PandasDataset(dataset_name="labels")
features = transactions.get_input_fields(numerical_fields=numerical_fields)
label = labels.get_label_field(label_field="is_fraud")

In [ ]:
tb.set_onnx_model("tensorflowmodel", onnx_model_bytes)
onnx_model = tb.ONNX(model_save_name="tensorflowmodel")

In [ ]:
deployed_model = onnx_model.deploy("onnx_model_tf", input=features, labels=label)

In [ ]:
deployed_model.add_metric("WindowedAUC")

In [ ]:
model_auc_scores = deployed_model.get_evaluation("WindowedAUC")
plt.plot([model_auc_score.metric for model_auc_score in model_auc_scores])